1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2)Сложить все новости в БД

In [134]:
import requests
from lxml import html
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime, date

In [135]:
#задаем хедер для видимости использования браузера
header = {'User_Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

#запускаем монго клиент с стандарными настройками
client = MongoClient('localhost', 27017)
#создаем новую бд 
db = client['News']
lenta = db.lenta
yandex = db.yandex
mail = db.mail


### lenta.ru

In [136]:
response = requests.get('https://lenta.ru/',
                            headers=header)
dom = html.fromstring(response.text)
data_lenta = []
items = dom.xpath("//section[contains(@class, 'top7')]//div[contains(@class, 'item')]")

for item in items:
    data = {}
    source = 'https://lenta.ru'
    name = item.xpath(".//h2//a//text | .//a//text()")
    date = item.xpath(".//h2//a//time//@datetime | .//a//time//@datetime")
    
    link = item.xpath(".//h2//a//@href | .//a//@href")
    if link[0][0] == '/':
        link = source + link[0]
    else:
        link = link[0]
        
    data['name'] = name[1].replace('\xa0', ' ')
    data['source'] = source
    data['link'] = link
    data['date'] = date[0]
    data_lenta.append(data)
    
pprint(data_lenta)

[{'date': ' 14:44,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/nezhy_papa/',
  'name': 'Командующий ВМС Украины пообещал «вернуть» Севастополь',
  'source': 'https://lenta.ru'},
 {'date': ' 15:19,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/venediktov_peskov/',
  'name': 'Венедиктов рассказал о жалобах Пескова после заражения '
          'коронавирусом',
  'source': 'https://lenta.ru'},
 {'date': ' 15:18,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/bridemotherinlaw/',
  'name': 'Свекровь захотела надеть платье невесты на свою свадьбу и была '
          'пристыжена',
  'source': 'https://lenta.ru'},
 {'date': ' 15:13,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/priority/',
  'name': 'Назван приоритет Путина',
  'source': 'https://lenta.ru'},
 {'date': ' 15:06,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/podsud/',
  'name': 'Россиянин пойдет под суд за убийство 92-летнего ветерана',
  'source': 'https://lenta.ru'}

### mail.ru

In [137]:
response = requests.get('https://news.mail.ru/',
                            headers=header)
dom = html.fromstring(response.text)
data_mail = []
items = dom.xpath("//div[@class='cols__inner']")

for i in items:
    if i.xpath(".//span[text()='Политика']"):
        break
        
items = i.xpath(".//li[@class='list__item'] | .//a[@class='newsitem__title link-holder']")

for item in items:
    data = {}
    name = item.xpath(".//span[@class='link__text']//text() | .//span[@class='newsitem__title-inner']//text()")
    link = item.xpath(".//a//@href | .//@href")
    
    m_response = requests.get(link[0], headers=header)
    m_dom = html.fromstring(m_response.text)
    
    date = m_dom.xpath("//span[@class='note__text breadcrumbs__text js-ago']//@datetime")
    date = date[0].replace('T', ' ')[:-9]
    
    sourse = m_dom.xpath("//a[@class='link color_gray breadcrumbs__link']//@href")[0]
    
    data['date'] = date
    data['sourse'] = sourse
    data['name'] = name[0]
    data['link'] = link[0]
    
    data_mail.append(data)
    
pprint(data_mail)

[{'date': '2020-07-06 07:03',
  'link': 'https://news.mail.ru/politics/42457414/',
  'name': 'Великобритания может ввести персональные санкции против граждан '
          'России',
  'sourse': 'http://www.kommersant.ru'},
 {'date': '2020-07-06 05:36',
  'link': 'https://news.mail.ru/politics/42456499/',
  'name': 'ЦБ поддержал инициативу предоставлять банковские услуги по '
          'водительским правам',
  'sourse': 'http://iz.ru/'},
 {'date': '2020-07-06 03:37',
  'link': 'https://news.mail.ru/politics/42456429/',
  'name': 'Украинский флот готовится к полномасштабной конфронтации с Россией',
  'sourse': 'http://www.ria.ru'},
 {'date': '2020-07-05 20:27',
  'link': 'https://news.mail.ru/politics/42455215/',
  'name': 'Один из обвиняемых в убийстве Флойда освобожден под залог',
  'sourse': 'http://iz.ru/'}]


### yandex

In [138]:
response = requests.get('https://yandex.ru/news/',
                            headers=header)
dom = html.fromstring(response.text)
data_ya = []
items = dom.xpath("//div[@aria-labelledby='personal_feed']//td[@class='stories-set__item']")

for item in items:
    data = {}
    main_link = 'https://yandex.ru'
    
    name = item.xpath(".//a//text()")[0]
    link = main_link + item.xpath(".//a//@href")[0]
    date = item.xpath(".//div[@class='story__date']//text()")[0][-5:] + " " + str(datetime.now().date())
    sourse = item.xpath(".//div[@class='story__date']//text()")[0][:-6]
    
    data['date'] = date
    data['sourse'] = sourse
    data['name'] = name
    data['link'] = link 
    
    data_ya.append(data)
    
pprint(data_ya)
    

[{'date': '23:03 2020-07-06',
  'link': 'https://yandex.ru/news/story/ESPCH_kommuniciroval_zhalobu_na_rassledovanie_ubijstva_Nemcova--a221ab061b113aa177cb6375870977e8?lr=80&lang=ru&stid=_J7m-BIQJIt81ZeHiSon&persistent_id=104863914&rubric=personal_feed&from=index',
  'name': 'ЕСПЧ коммуницировал жалобу на расследование убийства Немцова',
  'sourse': 'Интерфакс'},
 {'date': '23:12 2020-07-06',
  'link': 'https://yandex.ru/news/story/Kujvashev_prodlil_koronavirusnye_ogranicheniya_v_Sverdlovskoj_oblasti--be98caa76e45f4db10a56f38750bc7f2?lr=80&lang=ru&stid=-3Z5SBziONcfKr880zUl&persistent_id=104868828&rubric=personal_feed&from=index',
  'name': 'Куйвашев продлил коронавирусные ограничения в Свердловской области',
  'sourse': 'Екатеринбург Онлайн'},
 {'date': '23:05 2020-07-06',
  'link': 'https://yandex.ru/news/story/Matchi_Orenburga_s_CSKA_i_Rostovom_nachnutsya_pozzhe--3091951b9cca106cff34a4c51fddc2ab?lr=80&lang=ru&stid=j8a-OFGh6J7oFxjuPcwM&persistent_id=104873582&rubric=personal_feed&from=

In [139]:
yandex.insert_many(data_ya)
mail.insert_many(data_mail)
lenta.insert_many(data_lenta)

'2020-07-06'